In [1]:
import numpy as np
import pandas as pd

In [33]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2011.xlsx", encoding='utf8')

In [34]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,18h30,Sábado,21/05/2011,Atlético-MG,Atlético-PR,3x0,Atlético-MG,1ª Rodada,Arena do Jacaré
1,18h30,Sábado,21/05/2011,Flamengo,Avaí,4x0,Flamengo,1ª Rodada,Cláudio Moacyr
2,18h30,Sábado,21/05/2011,Ceará,Vasco,1x3,Vasco,1ª Rodada,Presidente Vargas
3,21h00,Sábado,21/05/2011,Santos,Internacional,1x1,-,1ª Rodada,Vila Belmiro
4,16h00,Domingo,22/05/2011,Palmeiras,Botafogo,1x0,Palmeiras,1ª Rodada,Teixeirão


** Get all clubes for championship **



In [35]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [36]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [37]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,atlético-mg
1,flamengo
2,ceará
3,santos
4,palmeiras
5,grêmio
6,coritiba
7,figueirense
8,fluminense
9,américa-mg


In [38]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [39]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    if c1 == u'grêmio prudente':
        pg -= 3
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,atlético-mg,45.0,38.0,13.0,6.0,19.0,50.0,60.0,-10.0
1,flamengo,61.0,38.0,15.0,16.0,7.0,59.0,47.0,12.0
2,ceará,39.0,38.0,10.0,9.0,19.0,47.0,64.0,-17.0
3,santos,53.0,38.0,15.0,8.0,15.0,55.0,55.0,0.0
4,palmeiras,50.0,38.0,11.0,17.0,10.0,43.0,39.0,4.0
5,grêmio,48.0,38.0,13.0,9.0,16.0,49.0,57.0,-8.0
6,coritiba,57.0,38.0,16.0,9.0,13.0,57.0,41.0,16.0
7,figueirense,58.0,38.0,15.0,13.0,10.0,46.0,45.0,1.0
8,fluminense,63.0,38.0,20.0,3.0,15.0,60.0,51.0,9.0
9,américa-mg,37.0,38.0,8.0,13.0,17.0,51.0,69.0,-18.0


In [40]:
reorderlist = [  "corinthians", "vasco", "fluminense", "flamengo", "internacional", "são paulo", 
                 "figueirense", "coritiba", "botafogo-rj", "santos", "palmeiras", "grêmio", 
                 "atlético-go", "Bahia", "atlético-mg", "cruzeiro", "atlético-pr", "ceará", 
                 "américa-mg", "avaí"
              ]
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
        elif (c == 'botafogo-rj') & (c1 == 'botafogo'):
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,corinthians,71.0,38.0,21.0,8.0,9.0,53.0,36.0,17.0
2,vasco,69.0,38.0,19.0,12.0,7.0,57.0,40.0,17.0
3,fluminense,63.0,38.0,20.0,3.0,15.0,60.0,51.0,9.0
4,flamengo,61.0,38.0,15.0,16.0,7.0,59.0,47.0,12.0
5,internacional,60.0,38.0,16.0,12.0,10.0,57.0,43.0,14.0
6,são paulo,59.0,38.0,16.0,11.0,11.0,57.0,46.0,11.0
7,figueirense,58.0,38.0,15.0,13.0,10.0,46.0,45.0,1.0
8,coritiba,57.0,38.0,16.0,9.0,13.0,57.0,41.0,16.0
9,botafogo,56.0,38.0,16.0,8.0,14.0,52.0,49.0,3.0
10,santos,53.0,38.0,15.0,8.0,15.0,55.0,55.0,0.0


** O Grêmio Prudente foi punido com a perda de 3 pontos pela escalação **
** irregular do zagueiro Paulão, na partida contra o Flamengo, em 23/05. **

** Por algum motivo que desconheço, na fonte dos dados o Atlético Paranaense tem GC 44 ao invés de 45, estão o SG está resultando em -1, mas na soma dos dados o saldo de gols -2. **